# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [325]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np

def clean_country_name(country):
    """Remove digits and text within parentheses from country names"""
    country = ''.join([i for i in country if not i.isdigit()]).strip()
    if '(' in country:
        country = country.split('(')[0].strip()
    return country

def generate_energy_df():
    """Reads the Energy Indicators Excel file and returns a DataFrame with the following columns:
       - Country
       - Energy Supply
       - Energy Supply per Capita
       - % Renewable
    """
    country_rename_energy = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
    
    energy = pd.read_excel('assets/Energy Indicators.xls', skiprows=17, skipfooter=38, usecols="C:F", na_values="...")
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy['Energy Supply'] *= 1000000

    energy['Country'] = energy['Country'].apply(clean_country_name)
    energy['Country'] = energy['Country'].replace(country_rename_energy)

    return energy

def get_GDP_df():
    """Reads the World Bank Excel file and returns a DataFrame with the following columns:
       - Country
       - 2006-2015 GDP values
    """
    country_rename_gpd = {
    "Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}
    gdp = pd.read_csv('assets/world_bank.csv', skiprows=4)
    gdp['Country Name'] = gdp['Country Name'].replace(country_rename_gpd)
    gdp = gdp.rename(columns={"Country Name": "Country"})
    cols = ['Country'] + [str(num) for num in range(2006, 2016)]
    gdp = gdp[cols]
    
    return gdp

def get_scimago_df():
    """Reads the Scimago Journal and Country Rank data for Energy Engineering and Power Technology and returns a DataFrame with the following columns:
       - Rank
       - Documents
       - Citable documents
       - Citations
       - Self-citations
       - Citations per document
       - H index
    """
    return pd.read_excel('assets/scimagojr-3.xlsx')

def answer_one():
    """Joins three datasets and returns the top 15 countries by Scimagojr 'Rank'"""
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    # Merge datasets
    merged_df = pd.merge(scimago_df, energy_df, on='Country', how='inner')
    merged_df = pd.merge(merged_df, gdp_df, on='Country', how='inner')

    # Set the index to be the country name
    merged_df.set_index('Country', inplace=True)

    # Select relevant columns and sort by rank
    final_df = merged_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 
                          'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + [str(num) for num in range(2006, 2016)]]
    final_df = final_df.sort_values('Rank')

    return final_df.head(15)

# Execute the function to get the result
df_result = answer_one()
df_result


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [285]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [286]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [328]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [329]:
def answer_two():
    """ Determines how many entries were lost when joining the datasets in Problem 1."""
    
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    # Merge datasets
    merged_df = pd.merge(scimago_df, energy_df, on='Country', how='inner')
    merged_df = pd.merge(merged_df, gdp_df[['Country'] + [str(year) for year in range(2006, 2016)]], on='Country', how='inner')

    # Number of unique countries after merging
    num_countries_merged = len(merged_df['Country'].unique())

    # Calculate the number of countries lost
    total_unique_countries_before = len(set(energy_df['Country'].unique()) | set(gdp_df['Country'].unique()) | set(scimago_df['Country'].unique()))
    countries_lost = total_unique_countries_before - num_countries_merged
    
    return countries_lost

df_result = answer_two()
df_result

158

In [289]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [290]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [291]:
def answer_three():
    """ Returns the top 15 countries for average GDP over the last 10 years."""
    
    df = answer_one()
    
    # Calculate the average GDP over the last 10 years and sort in descending order
    avg_gdp = df[[str(year) for year in range(2006, 2016)]].mean(axis=1)
    avg_gdp = avg_gdp.sort_values(ascending=False)
    
    return avg_gdp

df_result = answer_three()
df_result.head(15)

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [292]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [293]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [294]:
def answer_four():
    """ Calculates the GDP change over the 10 year span for the country with the 6th largest average GDP."""

    avg_gdp = answer_three()
    
    # Get the country with the 6th largest average GDP
    sixth_country = avg_gdp.index[5]
    df_one = answer_one()
    
    # Calculate the GDP change for the sixth country
    gdp_change = float(df_one.loc[sixth_country, '2015']) - float(df_one.loc[sixth_country, '2006'])
    
    return gdp_change

df_result = answer_four()
df_result

246702696075.3999

In [295]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [296]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [297]:
def answer_five():
    """ Calculates the mean energy supply per capita."""
    
    # Get the DataFrame from answer_one
    df_one = answer_one()
    
    # Calculate the mean energy supply per capita
    mean_energy_supply_per_capita = float(df_one['Energy Supply per Capita'].mean())
    
    return mean_energy_supply_per_capita

df_result = answer_five()
df_result


157.6

In [298]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [299]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [300]:
def answer_six():
    """ Finds the country with the highest % Renewable and the percentage."""
    # Get the DataFrame from answer_one
    df_one = answer_one()
    
    # Find the country with the highest % Renewable and the percentage
    max_renewable_country = df_one['% Renewable'].idxmax()
    max_renewable_percentage = df_one['% Renewable'].max()
    
    # Return the result as a tuple
    return (max_renewable_country, max_renewable_percentage)

df_result = answer_six()
df_result


('Brazil', np.float64(69.64803))

In [301]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [302]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [303]:
def answer_seven():
    """ Calculates the ratio of Self-Citations to Total Citations and finds the country with the highest ratio."""
    
    # Get the DataFrame from answer_one
    df_one = answer_one()
    
    # Calculate the ratio of Self-Citations to Total Citations
    df_one['Self-Citations Ratio'] = df_one['Self-citations'] / df_one['Citations']
    
    # Find the country with the highest ratio
    max_self_citations_ratio_country = df_one['Self-Citations Ratio'].idxmax()
    max_self_citations_ratio = df_one['Self-Citations Ratio'].max()
    
    # Return the result as a tuple
    return (max_self_citations_ratio_country, max_self_citations_ratio)

df_result = answer_seven()
df_result


('China', np.float64(0.6893126179389422))

In [304]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [305]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [306]:
def answer_eight():
    """ Finds the third most populous country according to the estimated population."""

    # Get the DataFrame from answer_one
    df_one = answer_one()

    # Calculate the estimated population
    df_one['Estimated Population'] = df_one['Energy Supply'] / df_one['Energy Supply per Capita']

    # Sort the DataFrame by estimated population
    df_sorted = df_one.sort_values(by='Estimated Population', ascending=False)

    # Find the third most populous country
    third_populous_country = df_sorted.iloc[2].name
    
    return third_populous_country

df_result = answer_eight()
df_result

'United States'

In [307]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [308]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [309]:
def answer_nine():
    """ Calculates the country with the highest Energy Supply per Capita."""
    # Get the DataFrame from answer_one
    df_one = answer_one()

    # Find the country with the highest Energy Supply per Capita
    df_one['Estimated Population'] = df_one['Energy Supply'] / df_one['Energy Supply per Capita']

    # Calculate the number of citable documents per person
    df_one['Citable Documents per Capita'] = df_one['Citable documents'] / df_one['Estimated Population']

    # Calculate the correlation between the number of citable documents per capita and energy supply per capita
    correlation = df_one['Citable Documents per Capita'].corr(df_one['Energy Supply per Capita'])
    
    return correlation

df_result = answer_nine()
df_result

np.float64(0.7940010435442946)

In [310]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [311]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [312]:
def answer_ten():
    """ Adds a new column to the DataFrame that indicates if the % Renewable is above the median."""

    # Get the DataFrame from answer_one
    df_one = answer_one()

    # Calculate the median % Renewable
    median_renewable = df_one['% Renewable'].median()

    # Add a new column to the DataFrame that indicates if the % Renewable is above the median
    df_one['high_renew'] = (df_one['% Renewable'] >= median_renewable).astype(int)

    # Sort the DataFrame by rank
    high_renew_series = df_one['high_renew'].sort_values(ascending=True)
    
    return high_renew_series

df_result = answer_ten()
df_result


Country
United States         0
Japan                 0
United Kingdom        0
India                 0
Iran                  0
Australia             0
South Korea           0
China                 1
Germany               1
Canada                1
Russian Federation    1
France                1
Spain                 1
Italy                 1
Brazil                1
Name: high_renew, dtype: int64

In [313]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [314]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [315]:
def answer_eleven():
    """ Calculates the number of countries with high renewable energy."""
    
    # Dictionary of continents
    ContinentDict  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }
    
    # Get the DataFrame from answer_one
    df_one = answer_one()
    
    # Calculate the median % Renewable
    df_one['Estimated Population'] = df_one['Energy Supply'] / df_one['Energy Supply per Capita']
    
    # Add a new column to the DataFrame that indicates if the % Renewable is above the median
    df_one['Continent'] = df_one.index.to_series().map(ContinentDict)
    
    # Calculate the number of countries with high renewable energy
    continent_stats = df_one.groupby('Continent')['Estimated Population'].agg(['size', 'sum', 'mean', 'std'])
    
    return continent_stats

df_result = answer_eleven()
df_result

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [316]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [317]:
# space for proffesor tests

## Problem 12

Cut _% Renewable_ into $5$ bins. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups?

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries._

In [318]:
def answer_twelve():
    """ Calculates the population estimate of each country in the top 15 countries by Scimagojr 'Rank'."""	
    
    # Dictionary of continents
    ContinentDict  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }
    
    # Get the DataFrame from answer_one
    df_one = answer_one()
    
    # Calculate the population estimate of each country
    df_one['Continent'] = df_one.index.to_series().map(ContinentDict)
    
    # Calculate the population estimate of each country
    df_one['% Renewable bins'] = pd.cut(df_one['% Renewable'], 5)
    
    # Group the DataFrame by Continent and % Renewable bins
    grouped = df_one.groupby(['Continent', '% Renewable bins']).size()
    
    return grouped

df_result = answer_twelve()
df_result

C:\Users\alejo\AppData\Local\Temp\ipykernel_32424\2853442922.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_one.groupby(['Continent', '% Renewable bins']).size()


Continent      % Renewable bins
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

In [319]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"

C:\Users\alejo\AppData\Local\Temp\ipykernel_32424\2853442922.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_one.groupby(['Continent', '% Renewable bins']).size()
C:\Users\alejo\AppData\Local\Temp\ipykernel_32424\2853442922.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_one.groupby(['Continent', '% Renewable bins']).size()


In [320]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [321]:
def answer_thirteen():
    """ Converts the Population Estimate series to a string with thousands separator."""
    # Get the DataFrame from answer_one
    df_one = answer_one()
    
    # Calculate the population estimate of each country
    df_one['Estimated Population'] = df_one['Energy Supply'] / df_one['Energy Supply per Capita']
    
    # Convert the population estimate to a string with thousands separator
    pop_est = df_one['Estimated Population'].apply(lambda x: "{:,.0f}".format(x).replace(",", "."))
    
    return pop_est

df_result = answer_thirteen()
df_result

Country
China                 1.367.645.161
United States           317.615.385
Japan                   127.409.396
United Kingdom           63.870.968
Russian Federation      143.500.000
Canada                   35.239.865
Germany                  80.369.697
India                 1.276.730.769
France                   63.837.349
South Korea              49.805.430
Italy                    59.908.257
Spain                    46.443.396
Iran                     77.075.630
Australia                23.316.017
Brazil                  205.915.254
Name: Estimated Population, dtype: object

In [322]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [323]:
# space for proffesor tests